<a href="https://colab.research.google.com/github/sarahgruetz/AluraChallenge_DS_2/blob/main/Alura_Challenge_semana_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 43 kB/s 
     |████████████████████████████████| 199 kB 47.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=635d9eb3011419002b37cc262537d8a389a16917d3d82b5ca863bbd01a91d73c
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Alura Challenge") \
    .getOrCreate()

In [3]:
import zipfile

zipfile.ZipFile('/content/drive/MyDrive/curso-spark/ChallengeAlura/semana-2.zip','r')\
        .extractall('/content/drive/MyDrive/curso-spark/ChallengeAlura/dados')

In [50]:
dataset = spark.read.parquet('/content/drive/MyDrive/curso-spark/ChallengeAlura/dados/dataset_transformado_parquet')

In [5]:
dataset.show(5)

+--------------------+-----+----------+---------+---------+--------------------+-------+------+------------+------------+-----------+----+------------+------------+----------+----+-----+------+
|                  id|andar|area_total|area_util|banheiros|     caracteristicas|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|      bairro|        zona|condominio|iptu| tipo| valor|
+--------------------+-----+----------+---------+---------+--------------------+-------+------+------------+------------+-----------+----+------------+------------+----------+----+-----+------+
|03a386b6-7ab8-4ef...|    0|        43|       43|        1|[Churrasqueira, A...|      2|  null|       Usado| Apartamento|Residencial|   1|    Realengo|  Zona Oeste|       285|null|Venda| 22999|
|1fe78d41-b8e0-4d2...|    0|        44|       44|        1|                  []|      2|     0|       Usado| Apartamento|Residencial|   0|       Irajá|  Zona Norte|       170|   0|Venda|110000|
|1fa1c1e5-e98c-433...|    4|  

## Seleção de Features

In [6]:
dataset\
  .select('tipo_anuncio')\
  .groupBy('tipo_anuncio')\
  .count()\
  .show()

+------------+-----+
|tipo_anuncio|count|
+------------+-----+
|       Usado|66562|
+------------+-----+



In [7]:
dataset\
  .select('tipo_unidade')\
  .groupBy('tipo_unidade')\
  .count()\
  .show()

+------------+-----+
|tipo_unidade|count|
+------------+-----+
| Apartamento|66562|
+------------+-----+



In [8]:
dataset\
  .select('tipo_uso')\
  .groupBy('tipo_uso')\
  .count()\
  .show()

+-----------+-----+
|   tipo_uso|count|
+-----------+-----+
|Residencial|66562|
+-----------+-----+



In [9]:
dataset\
  .select('tipo')\
  .groupBy('tipo')\
  .count()\
  .show()

+-----+-----+
| tipo|count|
+-----+-----+
|Venda|66562|
+-----+-----+



In [10]:
from pyspark.sql import functions as f

In [11]:
dataset\
  .select(f.count(f.when(dataset.area_total == dataset.area_util, True)).alias('mesmo_valor') )\
  .show()

+-----------+
|mesmo_valor|
+-----------+
|      55384|
+-----------+



In [12]:
dataset\
  .select(f.count(f.when(f.isnan('area_total') | f.isnull('area_total'), True)).alias('area_total') )\
  .show()

+----------+
|area_total|
+----------+
|      9194|
+----------+



In [13]:
dataset\
  .select(f.count(f.when(f.isnan('area_util') | f.isnull('area_util'), True)).alias('area_util') )\
  .show()

+---------+
|area_util|
+---------+
|        0|
+---------+



In [14]:
dataset\
  .select('bairro')\
  .groupBy('bairro')\
  .count()\
  .show()

+-------------------+-----+
|             bairro|count|
+-------------------+-----+
|             Cocotá|   16|
|       Tomás Coelho|   38|
|              Gávea|  597|
|            Ipanema| 2113|
|           Realengo|   46|
|      Gardênia Azul|    7|
|      Bento Ribeiro|   33|
|              Rocha|   38|
|Vicente de Carvalho|   86|
|         Manguinhos|    1|
|       Campo Grande|  639|
|        Jacarepaguá| 4437|
|               Leme|  234|
|           Botafogo| 3499|
|       Padre Miguel|   15|
|       Santo Cristo|  357|
|           Flamengo| 1378|
|  Engenho da Rainha|   25|
| Pedra de Guaratiba|   12|
|            Piedade|  104|
+-------------------+-----+
only showing top 20 rows



In [15]:
dataset\
    .select('bairro')\
    .distinct()\
    .count()

154

`id` -> código de identificação do anúncio. Não será utilizada no modelo.

`tipo_anuncio`, `tipo_unidade`, `tipo_uso`, `tipo` -> foram filtrados anteriormente e possuem apenas um registro distinto cada, portanto essas colunas também não serão utilizadas no modelo.

`area_total` e `area_util` -> as duas colunas quase sempre tem o mesmo valor. Além disso a coluna `area_total` tem diversos valores faltantes. Dessa forma será utilizada apenas a coluna `area_util` no modelo.

`bairro`-> possui uma quantidade muito grande de valores distintos, alguns com poucos registros. Não será utilizada no modelo.

In [51]:
dataset = dataset.drop('id','tipo_anuncio','tipo_unidade','tipo_uso','tipo','area_total', 'bairro')

## Convertendo tipo das colunas

In [17]:
dataset.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_util: string (nullable = true)
 |-- banheiros: long (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- quartos: long (nullable = true)
 |-- suites: long (nullable = true)
 |-- vaga: long (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: string (nullable = true)
 |-- iptu: string (nullable = true)
 |-- valor: string (nullable = true)



In [18]:
from pyspark.sql.types import DoubleType

In [52]:
dataset = dataset\
              .withColumn('area_util', dataset.area_util.cast(DoubleType()))\
              .withColumn('condominio', dataset.condominio.cast(DoubleType()))\
              .withColumn('iptu', dataset.iptu.cast(DoubleType()))\
              .withColumn('valor', dataset.valor.cast(DoubleType()))

In [21]:
dataset.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_util: double (nullable = true)
 |-- banheiros: long (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- quartos: long (nullable = true)
 |-- suites: long (nullable = true)
 |-- vaga: long (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: double (nullable = true)
 |-- iptu: double (nullable = true)
 |-- valor: double (nullable = true)



## Arrumando a coluna `caracteristicas`

In [22]:
from pyspark.sql import functions as f

In [ ]:
dataset = dataset.withColumn('caracteristicas',
                   f.when((f.size(f.col('caracteristicas')) == 0), f.lit(None)).otherwise(f.col("caracteristicas")))

In [ ]:
dataset.show(5)

In [53]:
caracteristicas = dataset.select(f.explode('caracteristicas').alias('caracteristicas')).distinct()
caracteristicas.show()

+------------------+
|   caracteristicas|
+------------------+
|Condomínio fechado|
|        Playground|
| Portão eletrônico|
|           Piscina|
|Animais permitidos|
|      Portaria 24h|
|          Elevador|
|          Academia|
|   Salão de festas|
|     Churrasqueira|
+------------------+



In [54]:
lista_caracteristicas = list(caracteristicas.toPandas()['caracteristicas'])
lista_caracteristicas

['Condomínio fechado',
 'Playground',
 'Portão eletrônico',
 'Piscina',
 'Animais permitidos',
 'Portaria 24h',
 'Elevador',
 'Academia',
 'Salão de festas',
 'Churrasqueira']

In [55]:
for caract in lista_caracteristicas:
  dataset = dataset.withColumn(caract, f.when(f.array_contains(dataset['caracteristicas'],caract),f.lit(1)).otherwise(f.lit(0)))

In [56]:
dataset.show(5)

+-----+---------+---------+--------------------+-------+------+----+------------+----------+------+--------+------------------+----------+-----------------+-------+------------------+------------+--------+--------+---------------+-------------+
|andar|area_util|banheiros|     caracteristicas|quartos|suites|vaga|        zona|condominio|  iptu|   valor|Condomínio fechado|Playground|Portão eletrônico|Piscina|Animais permitidos|Portaria 24h|Elevador|Academia|Salão de festas|Churrasqueira|
+-----+---------+---------+--------------------+-------+------+----+------------+----------+------+--------+------------------+----------+-----------------+-------+------------------+------------+--------+--------+---------------+-------------+
|    0|     43.0|        1|[Churrasqueira, A...|      2|  null|   1|  Zona Oeste|     285.0|  null| 22999.0|                 1|         1|                1|      0|                 1|           1|       0|       1|              1|            1|
|    0|     44.0|   

In [57]:
dataset = dataset.drop('caracteristicas')

## Lidando com os valores nulos

In [58]:
dataset\
  .select([f.count(f.when(f.isnan(c) | f.isnull(c), True)).alias(c) for c in dataset.columns])\
  .show()

+-----+---------+---------+-------+------+----+----+----------+----+-----+------------------+----------+-----------------+-------+------------------+------------+--------+--------+---------------+-------------+
|andar|area_util|banheiros|quartos|suites|vaga|zona|condominio|iptu|valor|Condomínio fechado|Playground|Portão eletrônico|Piscina|Animais permitidos|Portaria 24h|Elevador|Academia|Salão de festas|Churrasqueira|
+-----+---------+---------+-------+------+----+----+----------+----+-----+------------------+----------+-----------------+-------+------------------+------------+--------+--------+---------------+-------------+
|    0|        0|        0|      0|  5554|3017|   0|      2371|7199|    0|                 0|         0|                0|      0|                 0|           0|       0|       0|              0|            0|
+-----+---------+---------+-------+------+----+----+----------+----+-----+------------------+----------+-----------------+-------+------------------+-------

In [59]:
dataset = dataset\
    .select('*')\
    .na\
    .fill(0)

### Arrumando a coluna `zona`

In [61]:
dataset\
  .select('zona')\
  .groupBy('zona')\
  .count()\
  .show()

+------------+-----+
|        zona|count|
+------------+-----+
|  Zona Norte|11897|
|  Zona Oeste|32979|
|Zona Central| 1144|
|    Zona Sul|20531|
|            |   11|
+------------+-----+



In [62]:
dataset = dataset\
              .where(f.col('zona')!='')

In [63]:
dataset\
  .select('zona')\
  .groupBy('zona')\
  .count()\
  .show()

+------------+-----+
|        zona|count|
+------------+-----+
|  Zona Norte|11897|
|  Zona Oeste|32979|
|Zona Central| 1144|
|    Zona Sul|20531|
+------------+-----+



In [64]:
dataset = dataset.select("*").withColumn("id", f.monotonically_increasing_id())

In [66]:
dataset.show(5)

+-----+---------+---------+-------+------+----+------------+----------+------+--------+------------------+----------+-----------------+-------+------------------+------------+--------+--------+---------------+-------------+---+
|andar|area_util|banheiros|quartos|suites|vaga|        zona|condominio|  iptu|   valor|Condomínio fechado|Playground|Portão eletrônico|Piscina|Animais permitidos|Portaria 24h|Elevador|Academia|Salão de festas|Churrasqueira| id|
+-----+---------+---------+-------+------+----+------------+----------+------+--------+------------------+----------+-----------------+-------+------------------+------------+--------+--------+---------------+-------------+---+
|    0|     43.0|        1|      2|     0|   1|  Zona Oeste|     285.0|   0.0| 22999.0|                 1|         1|                1|      0|                 1|           1|       0|       1|              1|            1|  0|
|    0|     44.0|        1|      2|     0|   0|  Zona Norte|     170.0|   0.0|110000.0| 

In [68]:
zona = dataset\
          .groupBy('id')\
          .pivot('zona')\
          .agg(f.lit(1))\
          .na\
          .fill(0)
zona.show()

+-----+------------+----------+----------+--------+
|   id|Zona Central|Zona Norte|Zona Oeste|Zona Sul|
+-----+------------+----------+----------+--------+
|40395|           0|         0|         1|       0|
|53509|           0|         0|         1|       0|
| 8075|           0|         1|         0|       0|
|18628|           0|         0|         1|       0|
|29089|           0|         0|         0|       1|
| 2927|           0|         0|         0|       1|
| 7747|           0|         1|         0|       0|
|39598|           0|         1|         0|       0|
|64317|           0|         1|         0|       0|
|15371|           0|         0|         1|       0|
|42969|           0|         0|         1|       0|
|15375|           0|         1|         0|       0|
|19907|           0|         0|         0|       1|
|54039|           0|         0|         1|       0|
|37261|           0|         0|         1|       0|
| 9233|           0|         0|         1|       0|
|45166|     

In [69]:
dataset = dataset\
              .join(zona,'id', how='inner')

In [70]:
dataset.show(5)

+-----+-----+---------+---------+-------+------+----+----------+----------+------+---------+------------------+----------+-----------------+-------+------------------+------------+--------+--------+---------------+-------------+------------+----------+----------+--------+
|   id|andar|area_util|banheiros|quartos|suites|vaga|      zona|condominio|  iptu|    valor|Condomínio fechado|Playground|Portão eletrônico|Piscina|Animais permitidos|Portaria 24h|Elevador|Academia|Salão de festas|Churrasqueira|Zona Central|Zona Norte|Zona Oeste|Zona Sul|
+-----+-----+---------+---------+-------+------+----+----------+----------+------+---------+------------------+----------+-----------------+-------+------------------+------------+--------+--------+---------------+-------------+------------+----------+----------+--------+
|40395|    1|     74.0|        2|      2|     2|   1|Zona Oeste|       1.0|   1.0|1061791.0|                 1|         1|                1|      1|                 1|           1| 

In [71]:
dataset = dataset.drop('id','zona')

In [72]:
dataset.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_util: double (nullable = false)
 |-- banheiros: long (nullable = true)
 |-- quartos: long (nullable = true)
 |-- suites: long (nullable = true)
 |-- vaga: long (nullable = true)
 |-- condominio: double (nullable = false)
 |-- iptu: double (nullable = false)
 |-- valor: double (nullable = false)
 |-- Condomínio fechado: integer (nullable = false)
 |-- Playground: integer (nullable = false)
 |-- Portão eletrônico: integer (nullable = false)
 |-- Piscina: integer (nullable = false)
 |-- Animais permitidos: integer (nullable = false)
 |-- Portaria 24h: integer (nullable = false)
 |-- Elevador: integer (nullable = false)
 |-- Academia: integer (nullable = false)
 |-- Salão de festas: integer (nullable = false)
 |-- Churrasqueira: integer (nullable = false)
 |-- Zona Central: integer (nullable = true)
 |-- Zona Norte: integer (nullable = true)
 |-- Zona Oeste: integer (nullable = true)
 |-- Zona Sul: integer (nullable = true)



### Salvando dataset em parquet

In [74]:
dataset.write.parquet(
    path='/content/drive/MyDrive/curso-spark/ChallengeAlura/dados/parquet',
    mode='overwrite'
)

### Preparando os dados para os algoritmos do Spark MLib

In [75]:
from pyspark.ml.feature import VectorAssembler

In [76]:
dataset = dataset.withColumnRenamed('valor','label')

In [77]:
dataset.columns

['andar',
 'area_util',
 'banheiros',
 'quartos',
 'suites',
 'vaga',
 'condominio',
 'iptu',
 'label',
 'Condomínio fechado',
 'Playground',
 'Portão eletrônico',
 'Piscina',
 'Animais permitidos',
 'Portaria 24h',
 'Elevador',
 'Academia',
 'Salão de festas',
 'Churrasqueira',
 'Zona Central',
 'Zona Norte',
 'Zona Oeste',
 'Zona Sul']

In [78]:
X = [
    'andar',
    'area_util',
    'banheiros',
    'quartos',
    'suites',
    'vaga',
    'condominio',
    'iptu',
    'Condomínio fechado',
    'Playground',
    'Portão eletrônico',
    'Piscina',
    'Animais permitidos',
    'Portaria 24h',
    'Elevador',
    'Academia',
    'Salão de festas',
    'Churrasqueira',
    'Zona Central',
    'Zona Norte',
    'Zona Oeste',
    'Zona Sul' 
]

In [79]:
assembler = VectorAssembler(inputCols=X,outputCol='features')

In [80]:
dataset_modelo = assembler.transform(dataset).select('features','label')

In [81]:
dataset_modelo.show(10, truncate=False)

+-------------------------------------------------------------------------------------------------+---------+
|features                                                                                         |label    |
+-------------------------------------------------------------------------------------------------+---------+
|[1.0,74.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0]       |1061791.0|
|[0.0,188.0,5.0,3.0,3.0,2.0,1600.0,5390.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0]|1781000.0|
|(22,[0,1,2,3,5,6,8,10,12,19],[4.0,80.0,2.0,2.0,1.0,830.0,1.0,1.0,1.0,1.0])                       |450000.0 |
|(22,[1,2,3,20],[45.0,1.0,1.0,1.0])                                                               |65000.0  |
|(22,[1,2,3,6,7,13,16,21],[94.0,1.0,2.0,1336.0,368.0,1.0,1.0,1.0])                                |1130000.0|
|(22,[1,2,3,4,5,6,7,9,14,16,21],[150.0,3.0,3.0,1.0,2.0,2104.0,393.0,1.0,1.0,1.0,1.0])             |1700000.0|
|(22,[1,2,

### Separando dataset de treino e de teste

In [83]:
treino, teste = dataset_modelo.randomSplit([0.7, 0.3], seed=101)

### Regressão Linear

In [82]:
from pyspark.ml.regression import LinearRegression

In [84]:
lr = LinearRegression()

In [85]:
modelo_lr = lr.fit(treino)

In [86]:
previsoes_lr_treino = modelo_lr.transform(treino)

In [89]:
previsoes_lr_teste = modelo_lr.transform(teste)

In [87]:
from pyspark.ml.evaluation import RegressionEvaluator

In [88]:
evaluator = RegressionEvaluator()

In [90]:
print('Linear Regression')
print("="*30)
print("Dados de Treino")
print("="*30)
print("R²: %f" % evaluator.evaluate(previsoes_lr_treino, {evaluator.metricName: "r2"}))
print("RMSE: %f" % evaluator.evaluate(previsoes_lr_treino, {evaluator.metricName: "rmse"}))
print("")
print("="*30)
print("Dados de Teste")
print("="*30)
print("R²: %f" % evaluator.evaluate(previsoes_lr_teste, {evaluator.metricName: "r2"}))
print("RMSE: %f" % evaluator.evaluate(previsoes_lr_teste, {evaluator.metricName: "rmse"}))

Linear Regression
Dados de Treino
R²: 0.666697
RMSE: 853352.183180

Dados de Teste
R²: 0.662490
RMSE: 846439.252651


### Decision Tree